To-do</br>
load the data</br>
check for missing data</br>
do EDA, check the coorelation between each dat point</br>
encode and scale the data</br>
choose and build models</br>
evaluate the models</br>
do feature selection and emgineering</br>
re-train the model and stack it using stacking model or voting model</br>
clean the test data</br>
do what was done to the trainf data</br>
generate predictions using the ensembled model</br>
print out submissions.</br>


In [1]:
#installing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
#reading thre training data
train = pd.read_csv("train.csv")
train.head(2)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250


In [3]:
#reading the test dat
test = pd.read_csv("test.csv")
test.head(2)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes


In [4]:
#checking infromation about the data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [5]:
#for test
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


the catch here is how to encode the data properly so as not to cause any form of data leakage

<p>Checking for missing data</p>

In [6]:
#check for missing data
train.isna().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [7]:
#for the test data
test.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64

<p>reset index to id</p>

In [8]:
train.set_index('id', inplace=True)
train.head(2)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250


In [9]:
#for test
test.set_index('id', inplace=True)
test.head(2)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
id,,,,,,,,,,,
54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes


Some EDA babies

fro the feture engineering, i will divide the engine column further using excel <br/>
horsepower <br/>cylinder volume <br/>engine type<br/>cranshaft configuration <br/>turbo charging

using regex to divide the engine feature into more features to help the model understand the data better

In [10]:
# Sample dataset
engine_data = train['engine'].to_list()

# Regular expression pattern
pattern = re.compile(r"""
    (?P<horsepower>\d+\.?\d*)HP\s+          # Horsepower (optional)
    (?P<fuel_displacement>\d+\.?\d*)L\s+         # Displacement
    (?P<engine_configuration>Straight|V|H)\s*      # Engine configuration
    (?P<no_of_cylinders>\d+)\s*                   # Number of cylinders
    (Cylinder\s+Engine\s+)?                 # Optional "Cylinder Engine" text
    (?P<additional>[\w\s/]+)\s*           # Additional descriptors (optional)
    """, re.VERBOSE)

In [13]:
# Define additional feature patterns
feature_patterns = {
        'fuel_injection_type': r'\b(GDI|PDI|MPI|HCCI|CIS|EFI|CRDI|DI|SFI|MPFI|PFITBI|SPU|PFI|GTDI)\b',
        'crankshaft_configuration': r'\b(DOHC|SOHC|OHV|VVT|VVL)\b',
        'turbo_charging': r'\b(Turbo|Twin Turbo|Electric Turbo|Sequential Turbo)\b',
        'number_of_valves': r'\b(\d+V)\b',
    }
# Function to parse additional descriptors
def parse_additional(additional, feature_patterns):
    features = {}
    for feature, fp in feature_patterns.items():
        match = re.search(fp, additional, re.IGNORECASE)
        if match:
            features[feature] = match.group(0)

    return features


In [14]:
# Extract components and store them in a list of dictionaries
extracted_data = []
for engine in engine_data:
    match = pattern.search(engine)
    if match:
        data = match.groupdict()
        additional_features = parse_additional(data['additional'], feature_patterns)
        data.update(additional_features)
        extracted_data.append(data)

# Convert to a DataFrame
df = pd.DataFrame(extracted_data)

# Display the DataFrame
df.head(2)


,horsepower,fuel_displacement,engine_configuration,no_of_cylinders,additional
0,375.0,3.5,V,6,Gasoline Fuel
1,300.0,3.0,Straight,6,Gasoline Fuel


In [13]:
#adding the fuel type column
df['fuel_type'] = train['fuel_type']
df.head(2)

,horsepower,fuel_displacement,engine_configuration,no_of_cylinders,additional,fuel_injection_type,crankshaft_configuration,turbo_charging,number_of_valves,fuel_type
0,375.0,3.5,V,6,Gasoline Fuel,None,None,None,None,Gasoline
1,300.0,3.0,Straight,6,Gasoline Fuel,None,None,None,None,Gasoline


In [27]:
df.isnull().sum()

horsepower                      0
fuel_displacement               0
engine_configuration            0
no_of_cylinders                 0
additional                      0
fuel_injection_type         21080
crankshaft_configuration    21080
turbo_charging              21080
number_of_valves            21080
dtype: int64

data imputation

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21080 entries, 0 to 21079
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   horsepower                21080 non-null  object
 1   fuel_displacement         21080 non-null  object
 2   engine_configuration      21080 non-null  object
 3   no_of_cylinders           21080 non-null  object
 4   additional                21080 non-null  object
 5   fuel_injection_type       385 non-null    object
 6   crankshaft_configuration  0 non-null      object
 7   turbo_charging            0 non-null      object
 8   number_of_valves          0 non-null      object
 9   fuel_type                 21080 non-null  object
dtypes: object(10)
memory usage: 1.6+ MB


In [16]:
df['fuel_displacement'] = df.fuel_displacement.astype('float')
df['no_of_cylinders'] = df.no_of_cylinders.astype('int')
df['horsepower'] = df.horsepower.astype('float')
#df['number_of_valves'] = df.number_of_valves.astype('int')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21080 entries, 0 to 21079
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   horsepower                21080 non-null  float64
 1   fuel_displacement         21080 non-null  float64
 2   engine_configuration      21080 non-null  object 
 3   no_of_cylinders           21080 non-null  int32  
 4   additional                21080 non-null  object 
 5   fuel_injection_type       385 non-null    object 
 6   crankshaft_configuration  0 non-null      object 
 7   turbo_charging            0 non-null      object 
 8   number_of_valves          0 non-null      object 
 9   fuel_type                 21080 non-null  object 
dtypes: float64(2), int32(1), object(7)
memory usage: 1.5+ MB


In [18]:
# Function to determine the fuel injection type
def determine_fuel_injection(fuel_type, no_of_cylinders, fuel_displacement):
    if fuel_type == 'Diesel':
        if no_of_cylinders > 4:
            return 'CRDI'  # Common Rail Direct Injection for diesel engines with more than 4 cylinders
        else:
            return 'Direct Injection'  # Direct Injection for smaller diesel engines
    elif fuel_type == 'Gasoline':
        if fuel_displacement >= 3.0:
            return 'GDI'  # Gasoline Direct Injection for larger gasoline engines
        elif no_of_cylinders > 4:
            return 'MPFI'  # Multi-Point Fuel Injection for larger gasoline engines with more cylinders
        else:
            return 'PFI'  # Port Fuel Injection for smaller gasoline engines
    else:
        return 'None'

In [19]:

# Apply the function to fill in missing fuel injection types
df['fuel_injection_type'] = df.apply(
    lambda row: row['fuel_injection_type'] if row['fuel_injection_type'] else determine_fuel_injection(row['fuel_type'], row['no_of_cylinders'], row['fuel_displacement']),
    axis=1
)

In [20]:
# Display the DataFrame
df.head(2)

,horsepower,fuel_displacement,engine_configuration,no_of_cylinders,additional,fuel_injection_type,crankshaft_configuration,turbo_charging,number_of_valves,fuel_type
0,375.0,3.5,V,6,Gasoline Fuel,GDI,None,None,None,Gasoline
1,300.0,3.0,Straight,6,Gasoline Fuel,GDI,None,None,None,Gasoline


In [21]:
df.isnull().sum()

horsepower                      0
fuel_displacement               0
engine_configuration            0
no_of_cylinders                 0
additional                      0
fuel_injection_type             0
crankshaft_configuration    21080
turbo_charging              21080
number_of_valves            21080
fuel_type                       0
dtype: int64

In [22]:
#dropping columns with hefty missing values
df.drop(labels=['number_of_valves','turbo_charging' ,'crankshaft_configuration', 'fuel_type'], inplace=True, axis=1)
df.head(2)

,horsepower,fuel_displacement,engine_configuration,no_of_cylinders,additional,fuel_injection_type
0,375.0,3.5,V,6,Gasoline Fuel,GDI
1,300.0,3.0,Straight,6,Gasoline Fuel,GDI


In [23]:
df['additional'].value_counts()

additional
Gasoline Fuel                    19128
Gasoline/Mild Electric Hybrid     1023
Diesel Fuel                        385
Flex Fuel Capability               282
Gas/Electric Hybrid                220
Plug                                42
Name: count, dtype: int64

In [24]:
me = {'name':'chrity', 'age':22, 'level':400}
for i, j in me.items():
    print(i, j)

name chrity
age 22
level 400
